# Bounds on the number of samples of size $k$ from a population of size $n$ for various PRNGs and sampling algorithms

This notebook uses entropy-based bounds to find upper bounds on the number of samples that can be generated using a variety of algorithms for generating pseudo-random numbers (PRNs) and for turning sequences of PRNs into samples.

+ a hypothetical sampling algorithm that uses an optimal coding of samples to turn strings of $\log_2({n \choose k})$ bits into samples of size $k$
+ a common algorithm that involves assigning a PRN to each of the $n$ elements, then taking those elements assigned the smallest $k$ numbers to be the sample

PRNGs considered include [linear congruential generators](https://en.wikipedia.org/wiki/Linear_congruential_generator) (LCGs, including [RANDU](https://en.wikipedia.org/wiki/RANDU)), and the [Mersenne Twister](https://en.wikipedia.org/wiki/Mersenne_Twister).

[Kellie Ottoboni](http://www.stat.berkeley.edu/~kellieotto/) and [Philip B. Stark](www.stat.berkeley.edu/~stark)



In [1]:
from __future__ import division
import math
import numpy as np
import scipy as sp
from scipy.misc import comb, factorial
from scipy.optimize import brentq

In [4]:
def H(q):  # entropy of a Bernoulli(q) variable
    return -q*math.log(q, 2) - (1-q)*math.log(1-q, 2)
    

def comb_upper_bound(n, k):  # entropy upper bound on nCk
    q = k/n
    return 2**(n*H(q))


def comb_lower_bound(n, k): # entropy upper bound on nCk
    q = k/n
    return 2**(n*H(q))/(n+1)

In [3]:
# number of bits per period divided by log_2(choose(n,k))?
def sample_counter(n, k, period, word_length):
    return (period*word_length) > math.log(comb(n, k), 2)*comb(n, k)


def sample_counter_lb(n, k, period, word_length):
    ub_bits_for_all_samples = (n*H(k/n) - math.log(n+1, 2))*(2**(n*H(k/n)))/(n+1)
    return (period*word_length) > ub_bits_for_all_samples

# RANDU
The next few computations are specific to the RANDU PRNG, a particularly bad [LCG](https://en.wikipedia.org/wiki/Linear_congruential_generator) promulgated in the 1960s and widely copied.

In [5]:
randu_period = 2**29
randu_word_length = 32

In [6]:
n = 30
k = 20
can_do = True
while can_do:
    n = n+1
    can_do = sample_counter_lb(n, k, randu_period, randu_word_length)
    print n
print "can't do ",n

n = 30
k = 20
can_do = True
while can_do:
    n = n+1
    can_do = sample_counter(n, k, randu_period, randu_word_length)
    print n
print "can't do ",n

31
32
33
34
35
can't do  35
31
32
33
34
can't do  34


# Most efficient coding/a single period of the PRNG

Suppose we generate samples of size $k$ from a population of size $n$.
There are ${n \choose k}$ such samples.
The most efficient coding scheme requires $\log_2{n \choose k}$ bits to encode all possible samples.

A PRNG generates a "word" (typically an integer) using a fixed number of bits.
The period of the PRNG tells us the maximum number of words the PRNG can generate starting from a particular seed.
The period times the word length is the number of random bits that the PRNG can generate.
If this number is less than the number of bits per sample encoding times the number of samples, ${n \choose k}\log_2{n \choose k}$, then it is not possible to generate all possible samples in one period of the PRNG.

For a particular PRNG with known period and word length (typically 32 bits), we can fix $k$ and calculate the largest $n$ for which the PRNG can generate all possible samples.
Similarly, we can fix $n$ and find the largest $k$ for which the PRNG can generate all possible samples.
Note, when we vary $k$ it is only interesting for $1 \leq k \leq \lfloor{\frac{n}{2}}\rfloor$.
Without loss of generality, if $k > \lfloor{\frac{n}{2}}\rfloor$, we can think of the sampling as selecting individuals to *exclude* from the sample.

For large $n$, it's costly to compute binomial coefficients.
Instead, we use the following bound:

$$ \frac{2^{nH(k/n)}}{n+1} \leq {n \choose k} \leq 2^{nH(k/n)}$$

When dealing with the Mersenne Twister, we substitute the lower bound for ${n \choose k}$, giving overly optimistic estimates of maximum $n$.

## RANDU

In [7]:
k = [20, 50, 100, 500]
maxn = [0, 0, 0, 0]
i = 0
step_size = 10
for kk in k:
    can_do = True
    n = kk
    while can_do:
        n = n + step_size
        can_do = sample_counter_lb(n, kk, randu_period, randu_word_length)
#    print n
    
    count = lambda x: randu_period*randu_word_length - math.log(comb(x, kk), 2)*comb(x, kk)
    if kk==(n-step_size):
        llim = n - step_size + 1
    else:
        llim = n - step_size
    maxn[i] = np.trunc(brentq(count, llim, n))
    i = i + 1
    
print(maxn)

[33.0, 57.0, 105.0, 503.0]


In [8]:
n = [10, 20, 50, 100]
maxk = [0, 0, 0, 0]
i = 0
step_size = 1
for nn in n:
    can_do = True
    k = 0
    while can_do and k < nn/2:
        k = k + step_size
        can_do = sample_counter(nn, k, randu_period, randu_word_length)
    maxk[i] = k
    i = i + 1
    
print(maxk)

[5, 10, 9, 6]


## Mersenne Twister

In [9]:
mersenne_twister_period = (2**19937) - 1
mersenne_twister_word_length = 32

In [10]:
n = 100
k = 100
can_do = True
while can_do and n<1000000:
    n = n+1000
    can_do = sample_counter_lb(n, k, mersenne_twister_period, mersenne_twister_word_length)
    print n
print "can't do ",n

1100
2100
3100
4100
5100
6100
7100
8100
9100
10100
11100
12100
13100
14100
15100
16100
17100
18100
19100
20100
21100
22100
23100
24100
25100
26100
27100
28100
29100
30100
31100
32100
33100
34100
35100
36100
37100
38100
39100
40100
41100
42100
can't do  42100


In [11]:
k = [50, 100]
maxn = [0, 0]
i = 0
step_size = 10**4
for kk in k:
    can_do = True
    n = kk
    while can_do:
        n = n + step_size
        can_do = sample_counter_lb(n, kk, mersenne_twister_period, mersenne_twister_word_length)
    
    # This doesn't seem to work. I wonder if it's a problem with long int math.
#    count = lambda x: mersenne_twister_period*mersenne_twister_word_length - long((x*H(kk/x) - math.log(x+1, 2))*(2**(x*H(kk/x))/(x+1)))
#    maxn[i] = brentq(count, n-step_size, n)
    while not can_do:
        n = n - 1
        can_do = sample_counter_lb(n, kk, mersenne_twister_period, mersenne_twister_word_length)
    maxn[i] = n
    i = i + 1
    step_size = step_size/100
print maxn

[23434807, 41570.0]


# Naive Sampling Algorithm

The "naive sampling algorithm" is the algorithm that assigns a PRN to each item, then sorts and takes top k.
We ask:
- can all permutations occur?
- can all $nPk$ permutations occur?
- can all $nCk$ samples occur?

We look at RANDU and the Mersenne Twister.
Unfortunately, due to overflow errors, we don't have results for the Mersenne Twister yet.

In [14]:
# All permutations

k = np.array([10, 20, 50, 100])
all_perm = np.zeros((4, 2))
period_lengths = np.array([randu_period, mersenne_twister_period])
i = 0
for kk in k:
    j = 0
    for p in period_lengths:
        count = lambda nn: p - long((2**(nn*H(kk/nn))/(nn+1)))
        if count(10000)>0:
            all_perm[i, j] = -99
        else:
            all_perm[i, j] = brentq(count, kk+1, 10000)
        j = j + 1
    i = i + 1
print(all_perm)

[[  45.40777855  -99.        ]
 [  34.73680601  -99.        ]
 [  58.46911336  -99.        ]
 [ 106.60976355  -99.        ]]


In [15]:
# can all nPk permutations occur?

k = np.array([10, 20, 50, 100])
all_perm = np.zeros((4, 2))
period_lengths = np.array([randu_period, mersenne_twister_period])
i = 0
for kk in k:
    j = 0
    for p in period_lengths:
        count = lambda nn: p - long(factorial(nn)/factorial(kk))
        if count(150)>0:
            all_perm[i, j] = -99
        else:
            all_perm[i, j] = brentq(count, kk+1, 150)
        j = j + 1
    i = i + 1
print(all_perm)

[[  17.59068915  -99.        ]
 [  26.35785924  -99.        ]
 [  55.06263187  -99.        ]
 [ 104.34016714  -99.        ]]


In [16]:
# What's the largest n for which all nCk samples can occur?


k = np.array([10, 20, 50, 100])
all_perm = np.zeros((4, 2))
period_lengths = np.array([randu_period, mersenne_twister_period])
i = 0
for kk in k:
    j = 0
    for p in period_lengths:
        count = lambda nn: p - long(comb(nn,kk))
        if count(150)>0:
            all_perm[i, j] = -99
        else:
            all_perm[i, j] = brentq(count, kk+1, 150)
        j = j + 1
    i = i + 1
print(all_perm)

[[  38.42577637  -99.        ]
 [  32.92813183  -99.        ]
 [  57.35133302  -99.        ]
 [ 105.59084038  -99.        ]]


# testing out bounds when factorial overflows

In [19]:
factorial(41400)

array(inf)

In [13]:
print comb(41400,100)
print comb_upper_bound(41400, 100)
print comb_lower_bound(41400, 100)

inf
1.3025055052e+306
3.07187449635e+301
